# Sample Code

## 基礎建設

In [1]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

## 載入資料

In [2]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2022-01-02 12:12:19--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv.9’

All_Beauty.csv.9    100%[===================>]  14.78M  23.3MB/s    in 0.6s    

2022-01-02 12:12:20 (23.3 MB/s) - ‘All_Beauty.csv.9’ saved [15499476/15499476]

--2022-01-02 12:12:20--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz.9’

meta_All_Beauty.jso 100%[===================>]   9.85M  18.7MB/s    in 0.5s   

In [3]:
metadata = getDF('/content/meta_All_Beauty.json.gz')
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [4]:
metadata.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[]
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[]
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",[],"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [5]:
ratings.head()

,asin,reviewerID,overall,unixReviewTime
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800
2,0143026860,A1572GUYS7DGSR,4.0,1407628800
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200


將資料整理一下
把可以用的文字都擷取出來
一起做分析

In [6]:
metadata.description.apply(lambda x: len(x)).unique() 
def extract_string(ls):
  string = ''
  if len(ls) == 0:
    return ''
  for word in ls:
    string = string + word
  return string
metadata['description_text'] = metadata.description.apply(lambda x: extract_string(x))

In [7]:
metadata['all_text'] = metadata.description_text + metadata.title
metadata.all_text

0        Loud 'N Clear Personal Sound Amplifier allows ...
1        No7 Lift & Luminate Triple Action Serum 50ml b...
2        No7 Stay Perfect Foundation now stays perfect ...
3        Wella Koleston Perfect Hair Colour 44/44 Mediu...
4        Lacto Calamine Skin Balance Daily Nourishing L...
                               ...                        
32887    Barielle Pro Textured Grip Cuticle Nipper, Purple
32888    (Buy 3 Get 1 Free) Salon Perfect Eye Makeup Co...
32889    NOW D-Mannose 500 mg - 120 Veg Capsules (Pack ...
32890    Brand new and high quality<br> Enables fast vo...
32891    FESHFEN Scrunchy Scrunchies Synthetic Hair Bun...
Name: all_text, Length: 32892, dtype: object

## 資料整理

In [8]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')

## 資料切分

In [9]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

## 產生推薦

In [10]:
# RAM不足，會停止運作…
# 試著在文字上拿掉一些無義意的字

import re
def drop_words(string):
  string = re.sub('[^a-zA-Z ]*', '', string)
  string = re.sub(' +',' ', string)
  return string

metadata['all_text'] = metadata.all_text.apply(lambda x: drop_words(x))

In [26]:
# 重跑要執行的part 不然ram會不夠
del df
del tf
del tfidf_matrix
del similarity_matrix

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 計算商品用標題所表示的 tfidf 矩陣
df = metadata.drop_duplicates('title')
# tf = TfidfVectorizer(analyzer='word')
tf = TfidfVectorizer(analyzer='word',stop_words='english', min_df=0.001)
tfidf_matrix = tf.fit_transform(df['all_text'])

# 計算商品間的相似程度
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix = cosine_similarity(tfidf_matrix)
mapping = pd.Series(df.index,index = df['title'])

# 每個商品回傳 k 個最相近的商品
def recommend_item(item_input, k=2):
    try:
        item_index = mapping[item_input]
        similarity_score = list(enumerate(similarity_matrix[item_index]))
        similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
        similarity_score = similarity_score[:k]
        item_indices = [i[0] for i in similarity_score]
        return (df['asin'].iloc[item_indices].tolist())
    except:
        return []

# 利用使用者購買過的商品產生推薦
def recommend_items(items, k, top_df):
    if len(items) == 0:
      # 如果沒有購買記錄就給top10
      return top_df.asin.tolist() + ['new_cus'] # 我想用來判斷新客數
    res = []
    for d in items:
        res.extend(recommend_item(d, k))
    return res


In [28]:
def recommender(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    '''
    content-based
    '''
    ratings_trainings = training_data
    top_df = ratings_trainings[['asin','reviewerID']].groupby('asin', as_index=False).count()
    top_df = top_df.sort_values('reviewerID', ascending=False).iloc[:10,:]
    recommendations = {user: recommend_items(metadata[metadata['asin'].isin(ratings_trainings[ratings_trainings['reviewerID'] == user]['asin'].tolist())]['title'].tolist(), k, top_df) for user in users}
    return recommendations

ratings_by_user = recommender(ratings_trainings, users)
# ratings_by_user

## 結果評估

In [17]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

evaluate(ratings_testings_by_user, ratings_by_user)

0.08135593220338982

In [ ]:
# title : 0.08135593220338982
# all_text : 0.08135593220338982

In [20]:
old=[]
new=[]
for key, value in ratings_by_user.items():
  if 'new_cus' in value:
    new.append(key)
  else:
    old.append(key)
print('new', len(new), 'old', len(old))

new 546 old 38


In [30]:
ratings_testings_by_user_old = {key :ratings_testings_by_user[key] for key in old}
ratings_by_user_old = {key :ratings_by_user[key] for key in old}
evaluate(ratings_testings_by_user_old, ratings_by_user_old)

0.0

In [ ]:
# all text = 0.0